In [1]:
# write variable star observations in BAA format.
import os
from astropy.time import Time
jd_modifier = 2440000

In [2]:
def set_header():
    #  header text
    header = []
    header.append("Variable Star Observations BAAVSS format 2 version 1.2									\n")
    header.append("Year: 	2022										\n")
    header.append("Observer:  	C J Evans										\n")
    header.append('Address:	\"20 Windsor Crescent, Radyr, Cardiff, CF15 8AE\"								\n')
    header.append("Location:	51 36N  3 15W										\n")
    header.append("Instrument Code 1= NE,2= B8x30,3= B12x50, 4=Celestron C51251250							\n")
    header.append("					Data Input Screen						\n")
    header.append("					" + str(jd_modifier) + "						\n") # add this value to JD Dec column to get JD
    header.append("		U	T								\n")
    header.append("Star	Date	h	m	UT Dec	JD Dec	Estimate	Mag	Cl	Seq.	In	Notes\n")
    return(header)

In [3]:
def load_obs_csv(input_csv):
    '''
Load input file for write_baa_vss_upload.ipynb script.
required input format is:
    star, yyyy-mm-dd hh:mm, lightestimate, magnitude, obsclass, sequence, instrument, notes
    sequence can be left blank if there is a default sequence defined for that star
    notes can be left blank, but comma after instrument is required.      
    '''
    all_obs = []
    with open(input_csv) as f:
        lines = f.readlines()
    for line in lines:
        if line.find('#') == -1:
            all_obs.append(line)
    return(all_obs)

In [4]:
def convert_csv(all_obs):
    '''
    Convert the csv input format to baa vss format for upload
    all_obs is list of strings from input csv (one string per obs)
    returns all_obs_baa as a list of strings for csv input
    '''
    all_obs_out = []  #will be a list of formatted strings for baa input
    
    for obs_entry in all_obs:
        obs_field = obs_entry.split(',')
        t = Time(obs_field[1], format='iso', scale='utc')
        out_date = t.strftime('%d-%b-%Y') 
        out_hr = t.strftime('%H')
        out_min = t.strftime('%M')
        out_day = t.strftime('%d')
        out_month = t.strftime('%m')
        out_year = t.strftime('%Y')
        month_start = out_year + '-' + out_month + '-' + '01' + " 00:00"
        t_month_start = Time(month_start, format='iso', scale='utc')
        # baa's day of month counts from 1
        baa_day_of_month = "{:2.3f}".format(t.jd - t_month_start.jd + 1)
    #    print("{:2.3f}".format(t.jd - t_month_start.jd))
        out_jd = "{:5.3f}".format(t.jd-jd_modifier)
        obs_entry_out = [obs_field[0].strip().upper(), out_date, out_hr, out_min, baa_day_of_month, out_jd]
        for f in obs_field[2:]:
            obs_entry_out.append(f.strip())
        # check each obs for star, sequence.  add default sequence if known
        input_ok=input_check(obs_entry_out)

        line_out = ''
        if input_ok:
            new_obs_entry_out = [(e+'\t') for e in obs_entry_out]
            line_out = "".join(new_obs_entry_out)
            all_obs_out.append(line_out)
    
    return(all_obs_out)

In [5]:
def input_check(obs_to_check):
    '''
    check inputted values for validity (sequence, star,instruments)
    '''    
    instruments = [1,2,3,4]
    baa_seq = []
    baa_star = []
    with open('baa_vss_sequences.csv','r') as seq_file:
        for line in seq_file.readlines():
            baa_seq.append(line.replace('\n',''))
    with open('baa_vss_stars.csv', 'r') as star_file:
        for line in star_file.readlines():
            baa_star.append(line.replace('\n','').upper())     
    #with open('default_sequences.csv'):
    print(obs_to_check)
    if len(obs_to_check) == 12:
        print('length OK')
        len_ok = True
    if obs_to_check[0].upper() in baa_star:
        print('star OK')
        star_ok = True
    else:
        print('star ' + obs_to_check[0] + ' not found')
    if obs_to_check[9] in baa_seq:
        print('seq OK')
        seq_ok = True
    else:
        ###  Code for using default sequences would go in here
        print("Sequence not defined")
    all_ok = len_ok and star_ok and seq_ok
    print('Valid observation: ' + str(all_ok))
    return(all_ok)

In [7]:
# define input file here
input_path = "/home/john/astro/baa_vss_upload"
input_file = "baa_20221106b.csv"
input_full_path = os.path.join(input_path, input_file)

header = set_header()
obs = load_obs_csv(input_full_path)
obs = convert_csv(obs)
print(obs)

out_full_path = os.path.join(input_path,'test.txt')
# write data
with open(out_full_path, 'w') as out_file:
    for line in header:
        out_file.write(line)
    for obs_line in obs:
        out_file.write(obs_line + '\n')
out_file.close()

['Z UMA', '04-Nov-2022', '02', '34', '4.107', '19887.607', '=c', '7.5', '1', '217.02', '3', '']
length OK
star OK
seq OK
Valid observation: True
['T CEP', '04-Nov-2022', '03', '01', '4.126', '19887.626', '=g', '9.0', '1', '338.01', '3', '']
length OK
star OK
seq OK
Valid observation: True
['GAMMA CAS', '04-Nov-2022', '03', '06', '4.129', '19887.629', 'A-2', '2.0', '1', '64.01', '1', '']
length OK
star OK
seq OK
Valid observation: True
['EG AND', '04-Nov-2022', '03', '11', '4.133', '19887.633', '=B', '7.4', '1', '72.02', '3', '']
length OK
star OK
seq OK
Valid observation: True
['AD PER', '04-Nov-2022', '03', '22', '4.140', '19887.640', '=P', '8.5', '3', '344.01', '3', '']
length OK
star OK
seq OK
Valid observation: True
['TW AUR', '04-Nov-2022', '03', '37', '4.151', '19887.651', 'C(4)V(2)D', '8.1', '2', '354.01', '3', '']
length OK
star OK
seq OK
Valid observation: True
['ALPHA ORI', '04-Nov-2022', '04', '34', '4.190', '19887.690', 'C+1', '0.3', '1', '375.01', '1', '']
length OK
star O

In [ ]:
#############  OLD CODE ########################

In [ ]:
# format star, yyyy-mm-dd hh:mm, lightestimate, magnitude, obsclass, instrument, notes
all_obs = [ "tx dra,2022-11-04 02:30, n+2,7.9,3,3,low", \
        " z uma,2022-11-04 02:34, =c, 7.5,1,3,", \
          "eg and,2022-10-17 21:57,c-1,7.7,1,3,", \
    "EG AND,2022-10-17 21:57,C-1,7.7", \
    "GAMMA CAS,2022-10-17 21:17,=B,2.1", \
    "CH CYG,2022-10-17 21:48,A-2,6.7", \
    "P CYG,2022-10-17 21:44,=50,5.0", \
    "W CYG,2022-10-17 22:30,A-1,6.3", \
    "AH DRA,2022-10-17 22:08,=6,7.8", \
    "TX DRA,2022-10-17 21:00,K-2,7.2", \
    "ALPHA ORI,2022-10-18 05:30,=C,0.4", \
    "EG And,2021-12-06 20:38,b(1)v(1)c,7.5", \
    "EG And,2022-08-06 03:30,=a,7.0", \
    "gamma cas,2021-12-06 20:30,a(2)v(1)b,2.0"
          ]
all_obs_out = []
obs_entry_out = []

In [ ]:
# format star, yyyy-mm-dd hh:mm, lightestimate, magnitude, obsclass, sequence, instrument, notes
# sequence can be left blank if there is a default sequence defined for that star
all_obs = [ "tx dra,2022-11-04 02:30, n+2,7.9,3,,3,low", \
        " z uma,2022-11-04 02:34, =c, 7.5,1,,3,", \
          "eg and,2022-10-17 21:57,c-1,7.7,1,,3," ]
all_obs_out = []
obs_entry_out = []